#Assignment 3

#Submitted by

#Dhanish Ahammed MV
#MS21218

In [30]:
#unzip provided file and extract required
!unzip -o /content/1629377515-5e748a2d5fc288e9f69c5f86\ \(1\).zip


Archive:  /content/1629377515-5e748a2d5fc288e9f69c5f86 (1).zip
  inflating: mnist_train.csv         


In [31]:
#import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Question 1

Here you must read an input file. Each line contains 785 numbers (comma delimited): the first number in each row denotes the class label: 0 corresponds to digit 0, 1 corresponds to digit 1, etc. The rest of the values are the 784 pixel values between 0 and 255 correspondig to black and white images.  As a warm up  question, load the data.

For this problem you must write a function that takes a file path as an argument which contains  this data. Your function must return two values (x and y) that contains the data from the file as  described. Specifically, the first return value (x) must be a matrix where the rows are individual  examples of images, and the columns are individual pixels (n x 784 matrix). The second return value  must be a list/array of real numbers representing the labels of the examples (rows) in x.

eg:

1.0,0.0,1.0,0.0,....0.0,0.25,0.0,0.0

...

1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.96776

x = [

[1.0,0.0,1.0,0.0,....0.0,0.25,0.0,0.0]

...

[1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.96776]

]

y = [5,...,2]

#ANSWER 1

In [33]:
df = pd.read_csv("mnist_train.csv")
X = np.array(df.iloc[:,1:785])/255
Y = np.array(df["label"])

#Question 2

Implement the backpropagation algorithm in a zero hidden layer neural network (weights between input and output nodes). The output layer should be a softmax output over 10 classes corresponding to 10 classes of handwritten digits (e.g. An architecture: 784 > 10). Your backprop code should minimize the cross-entropy function for multi-class classification problems (categorical  cross entropy).



where j is the class label

This step should be done with a full step of gradient descent, not stochastic gradient descent or rmsprop. For this  problem you must write a function that takes as an input a matrix of x values, a list of y values (as  returned from problem 1), a weight matrix, and a learning rate and performs a single step of  backpropagation. You will need to do both a forward step with the inputs, and then a backward prop to  get the gradients. Return the updated weight matrix and bias in the same format as it was passed.

The list of weight matrices will be a list with 1 entry where the only entry is a matrix in the  format where the rows represent all of the outgoing weights for a neuron in the input layer and the  columns represent the weights for the incoming neurons. A specific row column index will give you the  weight for a neuron to neuron connection.

The list of bias vectors will be in the form where each entry in the list is a vector with the same  length as the first set of weights. (e.g. For an architecture of 784 > 10, there will be a single element list  with a vector of size 10).

#ANSWER 2

In [34]:
np.random.seed(100000) # Setting the seed for reproducibility

# One hot encoding the labels into 10 classes
Y_class = np.array([(i)*[0]+[1]+(9-i)*[0] for i in Y])

# Define the softmax function
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Define the forward pass function
def forward(x,w,b):
    z = np.dot(x, w) + b
    return softmax(z)

# Defining the training loop
def zero_layer_mlp(x,y,w,b,lr):
    num_iter = 100
    loss_list = []
    for iter in range(num_iter):
        # Forward pass
        y_pred = forward(x,w,b)

        # Calculating the cross entropy loss
        loss = -np.mean(y * np.log(y_pred))
        loss_list.append(loss)

        # Calculating the gradients
        grad_w = np.dot(x.T, y_pred - y) /X.shape[0]
        grad_b = np.mean(y_pred - y, axis=0)

        # Updating the weights and biases
        w = w-lr* grad_w
        b = b-lr* grad_b

        # Print the loss every 10 epochs
        if iter % 20 == 0:
            print(f"Epoch {iter}: loss = {loss:.4f}")

    return(w,b)

# Initializing the weights and biases
n = X.shape[1]
m = Y_class.shape[1]
w = np.random.randn(n,m) / np.sqrt(n)
b = np.zeros(m)

zero_layer_mlp(X,Y_class,w,b,0.1)

Epoch 0: loss = 0.2386
Epoch 20: loss = 0.1206
Epoch 40: loss = 0.0884
Epoch 60: loss = 0.0744
Epoch 80: loss = 0.0664


(array([[ 0.00256273, -0.06600526,  0.05410788, ..., -0.01516264,
         -0.03800385, -0.11119254],
        [ 0.00862694,  0.02516188, -0.02817639, ..., -0.02042698,
         -0.0258828 ,  0.07300685],
        [-0.06792594, -0.03998857,  0.0095788 , ..., -0.01201272,
         -0.02471961, -0.06152379],
        ...,
        [ 0.04452826,  0.01817994, -0.02118921, ..., -0.01404845,
         -0.01218868,  0.02777768],
        [ 0.04087397,  0.04071925, -0.01566047, ..., -0.01363192,
         -0.01624442,  0.02410428],
        [-0.06461566, -0.03825433,  0.02731732, ..., -0.04538838,
          0.04788461, -0.00829578]]),
 array([-0.05588458,  0.09295315, -0.02197461, -0.02456514,  0.02934742,
         0.05137224, -0.00675528,  0.04828548, -0.10449741, -0.00828126]))

#Question 3

Extend your code from problem 2 to support a single layer neural network with n hidden units (e.g. An  architecture: 784 > 10 > 10). These hidden units should be using sigmoid activations.

For this problem you must write a function that takes as an input a matrix of x values, a list of y  values (as returned from problem 1), list of weight matrices, a list of bias vectors, and a learning rate and performs a single step of backpropagation. You will need to do both a forward step with the inputs to get the outputs, and then a backward prop to get the gradients. Return the  updated weight matrix and bias in the same format as it was passed.

The list of weight matrices is a list with 2 entries where each entry in the list contains a single weight matrix as previously defined in problem 2. For a network with shape 784 > 10 > 10 the passed list  of weight matrices would look like this: [matrix with shape 784x10, matrix with shape 10x10]. Note:  though a hidden layer of size 10 is used as an example here, your code must be able to support a hidden  layer of dimension n.

The list of bias vectors will be in the form where each entry in the list is a vector with the same  length as the first set of weights. (e.g. For an architecture of 784 > 10 > 10, there will be a two element  list with an vector of size 10 and a vector of size 10)

#ANSWER 3

In [35]:
np.random.seed(100000)

# One hot encoding the labels into 10 classes
Y_class = np.array([(i)*[0]+[1]+(9-i)*[0] for i in Y])

# Defining the sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Defining the softmax function
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Defining the forward pass function
def forward(x,w1,w2,b1,b2):
    h = sigmoid(np.dot(x, w1) + b1)
    z = np.dot(h, w2) + b2
    return softmax(z)

# Defining the cost function
def cost(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred))

# Defining the training loop
def single_layer_mlp(x,y,W,B,lr):
    n_epochs = 100
    [w1,w2]=W
    [b1,b2]=B
    for epoch in range(n_epochs):
        # Forward pass
        h = sigmoid(np.dot(x, w1) + b1)
        y_pred = forward(x,w1,w2,b1,b2)

        # Calculating the cross entropy cost
        loss = cost(y, y_pred)

        # Calculating the gradients
        grad_z = y_pred - y
        grad_w2 = np.dot(h.T, grad_z) / x.shape[0]
        grad_b2 = np.mean(grad_z, axis=0)
        grad_h = np.dot(grad_z, w2.T) * h * (1 - h)
        grad_w1 = np.dot(x.T, grad_h) / x.shape[0]
        grad_b1 = np.mean(grad_h, axis=0)

        # Updating the weights and biases
        w2 = w2-lr * grad_w2
        b2 = b2-lr * grad_b2
        w1 = w1-lr * grad_w1
        b1 = b1-lr * grad_b1

        # Print the loss every 10 epochs
        if epoch % 20 == 0:
            print(f"Epoch {epoch}: loss = {loss:.4f}")

    W = [w1,w2]
    B = [b1,b2]
    return(W,B)

# Initializing the weights and biases
n = X.shape[1] # number of features
m = Y_class.shape[1] # number of classes
n_hu = 10  # number of hidden units

w1 = np.random.randn(n,n_hu) / np.sqrt(n)
b1 = np.zeros(n_hu)
w2 = np.random.randn(n_hu, m) / np.sqrt(n_hu)
b2 = np.zeros(m)

Weight = [w1,w2]
Bias = [b1,b2]

W,B=single_layer_mlp(X,Y_class,Weight,Bias,0.1)
print(W)
print(B)
print(W[0].shape,W[1].shape)
print(B[0].shape,B[1].shape)

Epoch 0: loss = 0.2393
Epoch 20: loss = 0.2220
Epoch 40: loss = 0.2133
Epoch 60: loss = 0.2054
Epoch 80: loss = 0.1978
[array([[ 0.00256273, -0.06600526,  0.05410788, ..., -0.01516264,
        -0.03800385, -0.11119254],
       [ 0.00862694,  0.02516188, -0.02817639, ..., -0.02042698,
        -0.0258828 ,  0.07300685],
       [-0.06792594, -0.03998857,  0.0095788 , ..., -0.01201272,
        -0.02471961, -0.06152379],
       ...,
       [ 0.04452826,  0.01817994, -0.02118921, ..., -0.01404845,
        -0.01218868,  0.02777768],
       [ 0.04087397,  0.04071925, -0.01566047, ..., -0.01363192,
        -0.01624442,  0.02410428],
       [-0.06461566, -0.03825433,  0.02731732, ..., -0.04538838,
         0.04788461, -0.00829578]]), array([[-0.20915034,  0.07774491,  0.45083931, -0.04617377, -0.41758153,
         0.63688707, -0.25639866, -0.12657251,  0.91276676, -0.30491518],
       [-0.12526245,  0.78158284,  0.05785329, -0.29687989, -0.07655511,
         0.35547708,  0.4779392 , -0.13224126,

#Question 4

Extend your code from problem 3 (use cross entropy error) and implement a multi-layer neural  network, starting with a simple architecture containing any number of hidden units in each layer (e.g. With  architecture: 784 > 10 > 10 > 10). These hidden units should be using sigmoid activations.

For this problem you must write a function that takes as an input a matrix of x values, a list of y  values (as returned from problem 1), list of weight matrices, a list of bias vectors, and a learning rate and  performs a single step of backpropagation. You will need to do both a forward step with the inputs to  get the outputs, and then a backward prop to get the gradients. Return the updated weight matrix and  bias in the same format as it was passed.

The list of weight matrices is a list with k entries where each entry in the list contains a single  weight matrix as previously defined in problem 2. For a network with shape 784 > 10 > 10 > 10 the  passed list of weight matrices would look like this: [matrix with shape 784x10, matrix with shape 10x10,  matrix with shape 10x10]. Note: though a hidden layer of size 10 is used as an example here, your code  must be able to support a hidden layer of dimension n.

The list of bias vectors will be in the form where each entry in the list is a vector with the same  length as the first set of weights. (e.g. For an architecture of 784 > 10 > 10, there will be a two element  list with an vector of size 10 and a vector of size 10)

#ANSWER 4

In [36]:
np.random.seed(100000)

# One hot encoding the labels into 10 classes
Y_class = np.array([(i)*[0]+[1]+(9-i)*[0] for i in Y])

# Defining the sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Defininf  the softmax function
def softmax(z):
    exps = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

# Making random initialized weights and bias for all layers
def initialize_weights(num_layers,n_hu):
    layer_sizes = [784]+num_layers*[n_hu]+[10]
    weight_bias = []
    for i in range(len(layer_sizes)-1):
        w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) / np.sqrt(layer_sizes[i])
        b = np.zeros((1, layer_sizes[i+1]))
        weight_bias.append((w,b))
    return weight_bias

# forward pass
def forward(x, weight):
    acti = [x]
    A = x
    for w, b in weight:
        z = np.dot(A, w) + b
        A = sigmoid(z)
        acti.append(A)
    y_h = softmax(z)
    return acti, y_h

# backward pass
def backward(x, y, acti, y_h,weight_bias, lamda):
    m = x.shape[0]
    dz = y_h - y
    dw = np.dot(acti[-2].T, dz)/m + (lamda * weight_bias[-1][0])/m
    db = np.sum(dz, axis=0, keepdims=True)/m
    grad = [(dw, db)]
    for i in range(len(weight_bias)-1, 0, -1):
        dA = np.dot(dz, weight_bias[i][0].T)
        dz = dA * acti[i] * (1 - acti[i])
        dw = np.dot(acti[i-1].T, dz) / m + (lamda * weight_bias[i-1][0]) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        grad.insert(0, (dw, db))
    return grad

def update(weight_bias, grad, lr):
    for i in range(len(weight_bias)):
        w, b = weight_bias[i]
        dw, db = grad[i]
        w = w- lr*dw
        b = b - lr* db
        weight_bias[i] = (w, b)
    return weight_bias

def train(x, y,num_layer=3,n_hu=20, num_iter=100, lr=0.1,lamda=0.01):
    weight_bias = initialize_weights(num_layer,n_hu)
    for epoch in range(num_iter):
        acti, y_h = forward(X, weight_bias)
        grad = backward(x,y, acti, y_h, weight_bias,lamda)
        weights = update(weight_bias, grad, lr)
        cost = -np.sum(y * np.log(y_h)) / x.shape[0]
        if epoch % 10 == 0:
            print("Epoch {}: loss = {}".format(epoch, cost))

    weigh = list(zip(*weights))[0]
    bia = list(zip(*weights))[1]
    return weigh,bia

num_layer=3 # Number of layers
n_hu=20 # number of hidden units
train(X,Y_class,3,20)

Epoch 0: loss = 2.438898925185021
Epoch 10: loss = 2.330165493663167
Epoch 20: loss = 2.307874446294917
Epoch 30: loss = 2.302199309110229
Epoch 40: loss = 2.3006168010502637
Epoch 50: loss = 2.3000525886326537
Epoch 60: loss = 2.2997393986846277
Epoch 70: loss = 2.2994873152276347
Epoch 80: loss = 2.299249216088691
Epoch 90: loss = 2.2990131994206084


((array([[ 0.00256273, -0.06600515,  0.05410779, ..., -0.02042694,
          -0.02588275,  0.07300673],
         [-0.06792583, -0.03998851,  0.00957879, ...,  0.01156511,
           0.02280227,  0.04486038],
         [ 0.04719391, -0.01432991,  0.04285393, ...,  0.00844747,
           0.00151294,  0.04007569],
         ...,
         [ 0.00047193,  0.05446774, -0.07609704, ..., -0.04234888,
          -0.03934672, -0.01621983],
         [ 0.03255095,  0.05758201,  0.10453215, ..., -0.01574548,
          -0.02864816,  0.07133386],
         [-0.06647086, -0.03183223,  0.01672802, ...,  0.02687025,
           0.01034864,  0.01300802]]),
  array([[ 2.08615438e-01, -1.71879157e-01,  3.99487741e-01,
           6.37495386e-02, -1.45592186e-01, -2.97372146e-03,
          -9.38569293e-02, -4.10408013e-01, -3.62697567e-01,
          -4.28099655e-01,  8.37920355e-02, -3.05314556e-01,
          -1.39726704e-01,  2.08299421e-01,  1.62317473e-01,
          -1.55511186e-02, -1.61954589e-02, -2.35213301

#Question 5

Extend your code from problem 4 to implement different activations functions which will be  passed as a parameter. In this problem all activations (except the final layer which should remain a  softmax) must be changed to the passed activation function.

#ANSWER 5

In [37]:
np.random.seed(100000)

# One hot encoding the labels into 10 classes
Y_class = np.array([(i)*[0]+[1]+(9-i)*[0] for i in Y])

# Defining the sigmoid activation functions
def activation_func(name,z):
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))
    def relu(z):
        return np.maximum(0, np.asarray(z, dtype=z.dtype))
    def tanh(z):
        return np.tanh(z)

    if name == "sigmoid":
        return(sigmoid(z))
    if name == "relu":
        return(relu(z))
    if name == "tanh":
        return(tanh(z))

# Defininf  the softmax function
def softmax(z):
    exps = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

# Making random initialized weights and bias for all layers
def initialize_weights(num_layers,n_hu):
    layer_sizes = [784]+num_layers*[n_hu]+[10]
    weight_bias = []
    for i in range(len(layer_sizes)-1):
        w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) / np.sqrt(layer_sizes[i])
        b = np.zeros((1, layer_sizes[i+1]))
        weight_bias.append((w,b))
    return weight_bias

# forward pass
def forward(x, weight):
    acti = [x]
    A = x
    for w, b in weight:
        z = np.dot(A, w) + b
        A = activation_func("relu",z)
        acti.append(A)
    y_h = softmax(z)
    return acti, y_h

# backward pass
def backward(x, y, acti, y_h,weight_bias, lamda):
    m = x.shape[0]
    dz = y_h - y
    dw = np.dot(acti[-2].T, dz)/m + (lamda * weight_bias[-1][0])/m
    db = np.sum(dz, axis=0, keepdims=True)/m
    grad = [(dw, db)]
    for i in range(len(weight_bias)-1, 0, -1):
        dA = np.dot(dz, weight_bias[i][0].T)
        dz = dA * acti[i] * (1 - acti[i])
        dw = np.dot(acti[i-1].T, dz) / m + (lamda * weight_bias[i-1][0]) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        grad.insert(0, (dw, db))
    return grad

def update(weight_bias, grad, lr):
    for i in range(len(weight_bias)):
        w, b = weight_bias[i]
        dw, db = grad[i]
        w = w- lr*dw
        b = b - lr* db
        weight_bias[i] = (w, b)
    return weight_bias

def train(x, y,num_layer=3,n_hu=20, num_iter=100, lr=0.1,lamda=0.01):
    weight_bias = initialize_weights(num_layer,n_hu)
    for epoch in range(num_iter):
        acti, y_h = forward(X, weight_bias)
        grad = backward(x,y, acti, y_h, weight_bias,lamda)
        weights = update(weight_bias, grad, lr)
        cost = -np.sum(y * np.log(y_h)) / x.shape[0]
        if epoch % 10 == 0:
            print("Epoch {}: loss = {}".format(epoch, cost))

    weigh = list(zip(*weights))[0]
    bia = list(zip(*weights))[1]
    return weigh,bia

num_layer=3 # Number of layers
n_hu=20 # number of hidden units
train(X,Y_class,3,20)

Epoch 0: loss = 2.31581705892741
Epoch 10: loss = 2.3084567546673544
Epoch 20: loss = 2.30250805329236
Epoch 30: loss = 2.2974877995018264
Epoch 40: loss = 2.2930880059302416
Epoch 50: loss = 2.2891039620862483
Epoch 60: loss = 2.2853918920046237
Epoch 70: loss = 2.281852707066694
Epoch 80: loss = 2.278414036192011
Epoch 90: loss = 2.2750200076842044


((array([[ 0.00256273, -0.06600515,  0.05410779, ..., -0.02042694,
          -0.02588275,  0.07300673],
         [-0.06792583, -0.03998851,  0.00957879, ...,  0.01156511,
           0.02280227,  0.04486038],
         [ 0.04719391, -0.01432991,  0.04285393, ...,  0.00844747,
           0.00151294,  0.04007569],
         ...,
         [ 0.00047193,  0.05446774, -0.07609704, ..., -0.04234888,
          -0.03934672, -0.01621983],
         [ 0.03255095,  0.05758201,  0.10453215, ..., -0.01574548,
          -0.02864816,  0.07133386],
         [-0.06647086, -0.03183223,  0.01672802, ...,  0.02687025,
           0.01034864,  0.01300802]]),
  array([[ 0.21070543, -0.1750123 ,  0.40045918,  0.06501277, -0.14409772,
          -0.00622635, -0.09408295, -0.409992  , -0.36268119, -0.42870973,
           0.08364539, -0.3040264 , -0.13741099,  0.20733159,  0.1611186 ,
          -0.01543212, -0.01626995, -0.23583664,  0.1019906 ,  0.32089352],
         [ 0.21232411,  0.22071442,  0.25728042,  0.2303305

#Question 6

Extend your code from problem 5 to implement momentum with your gradient descent. The  momentum value will be passed as a parameter. Your function should perform “epoch” number of  epochs and return the resulting weights.

#ANSWER 6

In [38]:
np.random.seed(100000)
# One hot encoding the labels into 10 classes
Y_class = np.array([(i)*[0]+[1]+(9-i)*[0] for i in Y])

# Defining the sigmoid activation functions
def activation_func(name, z):
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))
    def relu(z):
        return np.maximum(0, np.asarray(z, dtype=z.dtype))
    def tanh(z):
        return np.tanh(z)

    if name == "sigmoid":
        return sigmoid(z)
    if name == "relu":
        return relu(z)
    if name == "tanh":
        return tanh(z)

# Defininf the softmax function
def softmax(z):
    exps = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

# Making random initialized weights and bias for all layers
def initialize_weights(num_layers, n_hu):
    layer_sizes = [784] + num_layers * [n_hu] + [10]
    weight_bias = []
    for i in range(len(layer_sizes)-1):
        w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) / np.sqrt(layer_sizes[i])
        b = np.zeros((1, layer_sizes[i+1]))

        # Momentum terms
        vw = np.zeros_like(w)
        vb = np.zeros_like(b)
        weight_bias.append((w, b, vw, vb))

    return weight_bias

# Forward pass
def forward(x, weight):
    acti = [x]
    A = x
    for w, b, _, _ in weight:
        z = np.dot(A, w) + b
        A = activation_func("relu", z)
        acti.append(A)
    y_h = softmax(z)
    return acti, y_h

# Backward pass
def backward(x, y, acti, y_h, weight_bias, lamda):
    m = x.shape[0]
    dz = y_h - y
    dw = np.dot(acti[-2].T, dz) / m + (lamda * weight_bias[-1][0]) / m
    db = np.sum(dz, axis=0, keepdims=True) / m

    grad = [(dw, db)]
    for i in range(len(weight_bias)-1, 0, -1):
        dA = np.dot(dz, weight_bias[i][0].T)
        dz = dA * acti[i] * (1 - acti[i])
        dw = np.dot(acti[i-1].T, dz) / m + (lamda * weight_bias[i-1][0]) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        grad.insert(0, (dw, db))
    return grad

# Update weights and biases with momentum
def update(weight_bias, grad, lr, mr):
    for i in range(len(weight_bias)):
        w, b, vw, vb = weight_bias[i]
        dw, db = grad[i]
        vw = mr * vw - lr * dw
        vb = mr * vb - lr * db
        w = w + vw
        b = b + vb
        weight_bias[i] = (w, b, vw, vb)
    return weight_bias

# Training function
def train(x, y, num_layer=3, n_hu=20, num_iter=100, lr=0.1, lamda=0.01, mr=0.9):
    weight_bias = initialize_weights(num_layer, n_hu)
    for epoch in range(num_iter):
        acti, y_h = forward(x, weight_bias)
        grad = backward(x, y, acti, y_h, weight_bias, lamda)
        weights = update(weight_bias, grad, lr, mr)
        cost = -np.sum(y * np.log(y_h)) / x.shape[0]
        if epoch % 10 == 0:
            print("Epoch {}: loss = {}".format(epoch, cost))

    weigh = list(zip(*weights))[0]
    bia = list(zip(*weights))[1]
    return weigh, bia

# Number of layers
num_layer = 3
# Number of hidden units
n_hu = 20
# Training
train(X, Y_class, num_layer, n_hu)


Epoch 0: loss = 2.31581705892741
Epoch 10: loss = 2.2911414773180168
Epoch 20: loss = 2.2662832705573037
Epoch 30: loss = 2.2369846955694705
Epoch 40: loss = 2.1941402330572144
Epoch 50: loss = 2.1290399428652034
Epoch 60: loss = 2.0376323415987523
Epoch 70: loss = 1.9281442935356092
Epoch 80: loss = 1.8198586069405762
Epoch 90: loss = 1.718615663744177


((array([[ 0.00256269, -0.06600426,  0.05410706, ..., -0.02042667,
          -0.0258824 ,  0.07300574],
         [-0.06792491, -0.03998797,  0.00957866, ...,  0.01156496,
           0.02280197,  0.04485978],
         [ 0.04719328, -0.01432972,  0.04285335, ...,  0.00844735,
           0.00151292,  0.04007515],
         ...,
         [ 0.00047192,  0.054467  , -0.07609602, ..., -0.04234831,
          -0.03934619, -0.01621961],
         [ 0.03255051,  0.05758123,  0.10453074, ..., -0.01574526,
          -0.02864778,  0.0713329 ],
         [-0.06646997, -0.0318318 ,  0.0167278 , ...,  0.02686988,
           0.0103485 ,  0.01300784]]),
  array([[ 0.20647325, -0.17974039,  0.39869561,  0.06360451, -0.13121517,
          -0.00651895, -0.09443759, -0.41049219, -0.35461708, -0.42359786,
           0.09101434, -0.30310132, -0.12815793,  0.20678064,  0.16048677,
          -0.01602309, -0.00988201, -0.23308598,  0.10807936,  0.328783  ],
         [ 0.22083544,  0.22904533,  0.25441476,  0.2307350